In [1]:
# YOLOv5 🚀 by Ultralytics, GPL-3.0 license
"""
Train a YOLOv5 model on a custom dataset

Usage:
    $ python path/to/train.py --data coco128.yaml --weights yolov5s.pt --img 640
"""
from timeit import default_timer as timer
import pandas as pd
from collections import defaultdict
import pdb
import argparse
from label_converter import qt2yolo
from GPUtil import showUtilization as gpu_usage

from PIL.ImageFont import truetype
from label_converter import yolo2bcc_new
from train_with_bcc import convert_yolo2bcc, nn_predict, convert_cs_yolo2bcc
# YOLOBCC
from train_with_bcc import read_crowdsourced_labels, init_bcc_params, \
    init_nn_output, compute_param_confusion_matrices, init_metrics, update_bcc_metrics
from lib.BCCNet.VariationalInference.VB_iteration_yolo import VB_iteration as VBi_yolo
import logging
import math
import os
import random
import sys
import time
from copy import deepcopy
from pathlib import Path

import numpy as np
import torch
import torch.distributed as dist
import torch.nn as nn
import yaml
from torch.cuda import amp
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.optim import Adam, SGD, lr_scheduler
from tqdm import tqdm

# FILE = Path(__file__).absolute()
# sys.path.append(FILE.parents[0].as_posix())  # add yolov5/ to path

import val  # for end-of-epoch mAP
from models.experimental import attempt_load
from models.yolo import Model
from utils.autoanchor import check_anchors
from utils.datasets import create_dataloader
from utils.general import labels_to_class_weights, increment_path, labels_to_image_weights, init_seeds, \
    strip_optimizer, get_latest_run, check_dataset, check_file, check_git_status, check_img_size, \
    check_requirements, print_mutation, set_logging, one_cycle, colorstr, methods
from utils.downloads import attempt_download
from utils.loss import ComputeLoss
from utils.plots import plot_labels, plot_evolve
from utils.torch_utils import EarlyStopping, ModelEMA, de_parallel, intersect_dicts, select_device, \
    torch_distributed_zero_first
from utils.loggers.wandb.wandb_utils import check_wandb_resume
from utils.metrics import fitness
from utils.loggers import Loggers
from utils.callbacks import Callbacks



LOGGER = logging.getLogger(__name__)
LOCAL_RANK = int(os.getenv('LOCAL_RANK', -1))  # https://pytorch.org/docs/stable/elastic/run.html
RANK = int(os.getenv('RANK', -1))
WORLD_SIZE = int(os.getenv('WORLD_SIZE', 1))

In [2]:
def parse_opt(known=False):
    parser = argparse.ArgumentParser()
    parser.add_argument('--bcc', action='store_true', help='Whether to run YOLO with or without BCC.')
    parser.add_argument('--weights', type=str, default='yolov5s.pt', help='initial weights path')
    parser.add_argument('--cfg', type=str, default='', help='model.yaml path')
    parser.add_argument('--data', type=str, default='data/coco128.yaml', help='dataset.yaml path')
    parser.add_argument('--hyp', type=str, default='data/hyps/hyp.scratch.yaml', help='hyperparameters path')
    parser.add_argument('--epochs', type=int, default=300)
    parser.add_argument('--batch-size', type=int, default=16, help='total batch size for all GPUs')
    parser.add_argument('--imgsz', '--img', '--img-size', type=int, default=640, help='train, val image size (pixels)')
    parser.add_argument('--rect', action='store_true', help='rectangular training')
    parser.add_argument('--resume', nargs='?', const=True, default=False, help='resume most recent training')
    parser.add_argument('--nosave', action='store_true', help='only save final checkpoint')
    parser.add_argument('--noval', action='store_true', help='only validate final epoch')
    parser.add_argument('--noautoanchor', action='store_true', help='disable autoanchor check')
    parser.add_argument('--evolve', type=int, nargs='?', const=300, help='evolve hyperparameters for x generations')
    parser.add_argument('--bucket', type=str, default='', help='gsutil bucket')
    parser.add_argument('--cache', type=str, nargs='?', const='ram', help='--cache images in "ram" (default) or "disk"')
    parser.add_argument('--image-weights', action='store_true', help='use weighted image selection for training')
    parser.add_argument('--device', default='', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
    parser.add_argument('--multi-scale', action='store_true', help='vary img-size +/- 50%%')
    parser.add_argument('--single-cls', action='store_true', help='train multi-class data as single-class')
    parser.add_argument('--adam', action='store_true', help='use torch.optim.Adam() optimizer')
    parser.add_argument('--sync-bn', action='store_true', help='use SyncBatchNorm, only available in DDP mode')
    parser.add_argument('--workers', type=int, default=8, help='maximum number of dataloader workers')
    parser.add_argument('--project', default='runs/train', help='save to project/name')
    parser.add_argument('--entity', default=None, help='W&B entity')
    parser.add_argument('--name', default='exp', help='save to project/name')
    parser.add_argument('--exist-ok', action='store_true', help='existing project/name ok, do not increment')
    parser.add_argument('--quad', action='store_true', help='quad dataloader')
    parser.add_argument('--linear-lr', action='store_true', help='linear LR')
    parser.add_argument('--label-smoothing', type=float, default=0.0, help='Label smoothing epsilon')
    parser.add_argument('--upload_dataset', action='store_true', help='Upload dataset as W&B artifact table')
    parser.add_argument('--bbox_interval', type=int, default=-1, help='Set bounding-box image logging interval for W&B')
    parser.add_argument('--save_period', type=int, default=-1, help='Log model after every "save_period" epoch')
    parser.add_argument('--artifact_alias', type=str, default="latest", help='version of dataset artifact to be used')
    parser.add_argument('--local_rank', type=int, default=-1, help='DDP parameter, do not modify')
    parser.add_argument('--freeze', type=int, default=0, help='Number of layers to freeze. backbone=10, all=24')
    parser.add_argument('--patience', type=int, default=300, help='EarlyStopping patience (epochs)')
    opt = parser.parse_known_args()[0] if known else parser.parse_args("")
    return opt

## Main

In [3]:
data_name = 'toy'
opt = parse_opt()
opt.data = f'data/{data_name}.yaml' # Does not make any difference!
data_dict_path = f'../../datasets/{data_name}'
opt.exist_ok = True
opt.cache = None
opt.workers = 0
opt.batch_size = 10 # Change this to number of train images

# Checks
set_logging(RANK)
if RANK in [-1, 0]:
    print(colorstr('train: ') + ', '.join(f'{k}={v}' for k, v in vars(opt).items()))
#     check_git_status()
#     check_requirements(requirements=FILE.parent / 'requirements.txt', exclude=['thop'])

# Resume
if opt.resume and not check_wandb_resume(opt) and not opt.evolve:  # resume an interrupted run
    ckpt = opt.resume if isinstance(opt.resume, str) else get_latest_run()  # specified or most recent path
    assert os.path.isfile(ckpt), 'ERROR: --resume checkpoint does not exist'
    with open(Path(ckpt).parent.parent / 'opt.yaml') as f:
        opt = argparse.Namespace(**yaml.safe_load(f))  # replace
    opt.cfg, opt.weights, opt.resume = '', ckpt, True  # reinstate
    LOGGER.info(f'Resuming training from {ckpt}')
else:
    opt.data, opt.cfg, opt.hyp = check_file(opt.data), check_file(opt.cfg), check_file(opt.hyp)  # check files
    assert len(opt.cfg) or len(opt.weights), 'either --cfg or --weights must be specified'
    if opt.evolve:
        opt.project = 'runs/evolve'
        opt.exist_ok = opt.resume
    opt.save_dir = str(increment_path(Path(opt.project) / opt.name, exist_ok=opt.exist_ok))

# DDP mode
device = select_device(opt.device, batch_size=opt.batch_size)
if LOCAL_RANK != -1:
    from datetime import timedelta
    assert torch.cuda.device_count() > LOCAL_RANK, 'insufficient CUDA devices for DDP command'
    assert opt.batch_size % WORLD_SIZE == 0, '--batch-size must be multiple of CUDA device count'
    assert not opt.image_weights, '--image-weights argument is not compatible with DDP training'
    assert not opt.evolve, '--evolve argument is not compatible with DDP training'
    torch.cuda.set_device(LOCAL_RANK)
    device = torch.device('cuda', LOCAL_RANK)
    dist.init_process_group(backend="nccl" if dist.is_nccl_available() else "gloo")
hyp = opt.hyp
callbacks=Callbacks()

YOLOv5 🚀 ed16fb7 torch 1.9.0 CPU



train: bcc=False, weights=yolov5s.pt, cfg=, data=data/toy.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=300, batch_size=10, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=0, project=runs/train, entity=None, name=exp, exist_ok=True, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0, patience=300


## Train

In [4]:
save_dir, epochs, batch_size, weights, single_cls, evolve, data, cfg, resume, noval, nosave, workers, freeze, = \
    Path(opt.save_dir), opt.epochs, opt.batch_size, opt.weights, opt.single_cls, opt.evolve, opt.data, opt.cfg, \
    opt.resume, opt.noval, opt.nosave, opt.workers, opt.freeze
torchMode = True
# Directories
w = save_dir / 'weights'  # weights dir
w.mkdir(parents=True, exist_ok=True)  # make dir
last, best = w / 'last.pt', w / 'best.pt'

# Hyperparameters
if isinstance(hyp, str):
    with open(hyp) as f:
        hyp = yaml.safe_load(f)  # load hyps dict
LOGGER.info(colorstr('hyperparameters: ') + ', '.join(f'{k}={v}' for k, v in hyp.items()))

# Save run settings
with open(save_dir / 'hyp.yaml', 'w') as f:
    yaml.safe_dump(hyp, f, sort_keys=False)
with open(save_dir / 'opt.yaml', 'w') as f:
    yaml.safe_dump(vars(opt), f, sort_keys=False)
data_dict = {
    'path': data_dict_path,  # dataset root dir
    'train': 'images/train',  # train images (relative to 'path') 128 images
    'val': 'images/val',  # val images (relative to 'path') 128 images
    'test': 'images/test', # test images (optional)
    'nc': 2,  # number of classes
    'names': ['bone-loss', 'dental-caries']  # class names
}
for x in ['train', 'val', 'test']:
    data_dict[x] = os.path.join(data_dict['path'], data_dict[x])

# Loggers
if RANK in [-1, 0]:
    loggers = Loggers(save_dir, weights, opt, hyp, LOGGER)  # loggers instance
    if loggers.wandb:
        data_dict = loggers.wandb.data_dict
        if resume:
            weights, epochs, hyp = opt.weights, opt.epochs, opt.hyp

    # Register actions
    for k in methods(loggers):
        callbacks.register_action(k, callback=getattr(loggers, k))

# Config
plots = not evolve  # create plots
cuda = device.type != 'cpu'
init_seeds(1 + RANK)
with torch_distributed_zero_first(RANK):
    data_dict = data_dict or check_dataset(data)  # check if None
train_path, val_path = data_dict['train'], data_dict['val']
nc = 1 if single_cls else int(data_dict['nc'])  # number of classes
names = ['item'] if single_cls and len(data_dict['names']) != 1 else data_dict['names']  # class names
assert len(names) == nc, f'{len(names)} names found for nc={nc} dataset in {data}'  # check
is_coco = data.endswith('coco.yaml') and nc == 80  # COCO dataset

# Model
pretrained = weights.endswith('.pt')
if pretrained:
    with torch_distributed_zero_first(RANK):
        weights = attempt_download(weights)  # download if not found locally
    ckpt = torch.load(weights, map_location=device)  # load checkpoint
    model = Model(cfg or ckpt['model'].yaml, ch=3, nc=nc, anchors=hyp.get('anchors')).to(device)  # create
    exclude = ['anchor'] if (cfg or hyp.get('anchors')) and not resume else []  # exclude keys
    csd = ckpt['model'].float().state_dict()  # checkpoint state_dict as FP32
    csd = intersect_dicts(csd, model.state_dict(), exclude=exclude)  # intersect
    model.load_state_dict(csd, strict=False)  # load
    LOGGER.info(f'Transferred {len(csd)}/{len(model.state_dict())} items from {weights}')  # report
else:
    model = Model(cfg, ch=3, nc=nc, anchors=hyp.get('anchors')).to(device)  # create

# Freeze
freeze = [f'model.{x}.' for x in range(freeze)]  # layers to freeze
for k, v in model.named_parameters():
    v.requires_grad = True  # train all layers
    if any(x in k for x in freeze):
        print(f'freezing {k}')
        v.requires_grad = False

# Optimizer
nbs = 64  # nominal batch size
accumulate = max(round(nbs / batch_size), 1)  # accumulate loss before optimizing
hyp['weight_decay'] *= batch_size * accumulate / nbs  # scale weight_decay
LOGGER.info(f"Scaled weight_decay = {hyp['weight_decay']}")

g0, g1, g2 = [], [], []  # optimizer parameter groups
for v in model.modules():
    if hasattr(v, 'bias') and isinstance(v.bias, nn.Parameter):  # bias
        g2.append(v.bias)
    if isinstance(v, nn.BatchNorm2d):  # weight (no decay)
        g0.append(v.weight)
    elif hasattr(v, 'weight') and isinstance(v.weight, nn.Parameter):  # weight (with decay)
        g1.append(v.weight)

if opt.adam:
    optimizer = Adam(g0, lr=hyp['lr0'], betas=(hyp['momentum'], 0.999))  # adjust beta1 to momentum
else:
    optimizer = SGD(g0, lr=hyp['lr0'], momentum=hyp['momentum'], nesterov=True)

optimizer.add_param_group({'params': g1, 'weight_decay': hyp['weight_decay']})  # add g1 with weight_decay
optimizer.add_param_group({'params': g2})  # add g2 (biases)
LOGGER.info(f"{colorstr('optimizer:')} {type(optimizer).__name__} with parameter groups "
            f"{len(g0)} weight, {len(g1)} weight (no decay), {len(g2)} bias")
del g0, g1, g2

# Scheduler
if opt.linear_lr:
    lf = lambda x: (1 - x / (epochs - 1)) * (1.0 - hyp['lrf']) + hyp['lrf']  # linear
else:
    lf = one_cycle(1, hyp['lrf'], epochs)  # cosine 1->hyp['lrf']
scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lf)  # plot_lr_scheduler(optimizer, scheduler, epochs)

# EMA
ema = ModelEMA(model) if RANK in [-1, 0] else None

# Resume
start_epoch, best_fitness = 0, 0.0
if pretrained:
    # Optimizer
    if ckpt['optimizer'] is not None:
        optimizer.load_state_dict(ckpt['optimizer'])
        best_fitness = ckpt['best_fitness']

    # EMA
    if ema and ckpt.get('ema'):
        ema.ema.load_state_dict(ckpt['ema'].float().state_dict())
        ema.updates = ckpt['updates']

    # Epochs
    start_epoch = ckpt['epoch'] + 1
    if resume:
        assert start_epoch > 0, f'{weights} training to {epochs} epochs is finished, nothing to resume.'
    if epochs < start_epoch:
        LOGGER.info(f"{weights} has been trained for {ckpt['epoch']} epochs. Fine-tuning for {epochs} more epochs.")
        epochs += ckpt['epoch']  # finetune additional epochs

    del ckpt, csd

# Image sizes
gs = max(int(model.stride.max()), 32)  # grid size (max stride)
nl = model.model[-1].nl  # number of detection layers (used for scaling hyp['obj'])
imgsz = check_img_size(opt.imgsz, gs, floor=gs * 2)  # verify imgsz is gs-multiple

# DP mode
if cuda and RANK == -1 and torch.cuda.device_count() > 1:
    logging.warning('DP not recommended, instead use torch.distributed.run for best DDP Multi-GPU results.\n'
                    'See Multi-GPU Tutorial at https://github.com/ultralytics/yolov5/issues/475 to get started.')
    model = torch.nn.DataParallel(model)

# SyncBatchNorm
if opt.sync_bn and cuda and RANK != -1:
    model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model).to(device)
    LOGGER.info('Using SyncBatchNorm()')

#     print("*** GPU Usage before reading data")
#     gpu_usage()
# Trainloader
train_loader, dataset = create_dataloader(train_path, imgsz, batch_size // WORLD_SIZE, gs, single_cls,
                                          hyp=hyp, augment=False, cache=opt.cache, rect=opt.rect, rank=RANK,
                                          workers=workers, image_weights=opt.image_weights, quad=opt.quad,
                                          prefix=colorstr('train: '))
# print("*** GPU Usage after reading data")
# gpu_usage()
# YOLOBCC
# TODO: Verify the following (is nl == ng?):
n_grid_choices, n_anchor_choices = model.model[-1].nl, model.model[-1].na
grid_ratios = model.model[-1].stride.cpu().detach().numpy() / imgsz
cstargets_all = read_crowdsourced_labels(data)
cstargets_all_bcc = convert_cs_yolo2bcc(cstargets_all, n_anchor_choices, nc, grid_ratios)
# cstargets = cstargets_all['train']
cstargets_bcc = torch.tensor(cstargets_all_bcc['train']) if torchMode else cstargets_all_bcc['train']
# print("*** GPU Usage after reading crowdsourced labels")
# gpu_usage()
mlc = int(np.concatenate(dataset.labels, 0)[:, 0].max())  # max label class
nb = len(train_loader)  # number of batches
assert mlc < nc, f'Label class {mlc} exceeds nc={nc} in {data}. Possible class labels are 0-{nc - 1}'

# Process 0
if RANK in [-1, 0]:
    val_loader, val_dataset = create_dataloader(val_path, imgsz, batch_size // WORLD_SIZE * 2, gs, single_cls,
                                   hyp=hyp, cache=None if noval else opt.cache, rect=True, rank=-1,
                                   workers=workers, pad=0.5,
                                   prefix=colorstr('val: '))

    if not resume:
        labels = np.concatenate(dataset.labels, 0)
        # c = torch.tensor(labels[:, 0])  # classes
        # cf = torch.bincount(c.long(), minlength=nc) + 1.  # frequency
        # model._initialize_biases(cf.to(device))
        if plots:
            plot_labels(labels, names, save_dir)

        # Anchors
        if not opt.noautoanchor:
            check_anchors(dataset, model=model, thr=hyp['anchor_t'], imgsz=imgsz)
        model.half().float()  # pre-reduce anchor precision

    callbacks.on_pretrain_routine_end()

# DDP mode
if cuda and RANK != -1:
    model = DDP(model, device_ids=[LOCAL_RANK], output_device=LOCAL_RANK)

# Model parameters
hyp['box'] *= 3. / nl  # scale to layers
hyp['cls'] *= nc / 80. * 3. / nl  # scale to classes and layers
hyp['obj'] *= (imgsz / 640) ** 2 * 3. / nl  # scale to image size and layers
hyp['label_smoothing'] = opt.label_smoothing
model.nc = nc  # attach number of classes to model
model.hyp = hyp  # attach hyperparameters to model
model.class_weights = labels_to_class_weights(dataset.labels, nc).to(device) * nc  # attach class weights
model.names = names

# Start training
t0 = time.time()
nw = max(round(hyp['warmup_epochs'] * nb), 1000)  # number of warmup iterations, max(3 epochs, 1k iterations)
# nw = min(nw, (epochs - start_epoch) / 2 * nb)  # limit warmup to < 1/2 of training
last_opt_step = -1
maps = np.zeros(nc)  # mAP per class
results = (0, 0, 0, 0, 0, 0, 0)  # P, R, mAP@.5, mAP@.5-.95, val_loss(box, obj, cls)
scheduler.last_epoch = start_epoch - 1  # do not move
scaler = amp.GradScaler(enabled=cuda)
stopper = EarlyStopping(patience=opt.patience)
compute_loss = ComputeLoss(model)  # init loss class
# YOLOBCC
bcc_params = init_bcc_params()
bcc_params['n_epoch'] = epochs
batch_pcm = {k: torch.tensor(v).to(device) if torchMode else v for k, v in compute_param_confusion_matrices(bcc_params).items()}
# pred0_bcc = init_nn_output(dataset.n, grid_ratios, n_anchor_choices, bcc_params)
bcc_metrics = init_metrics(bcc_params['n_epoch'])
LOGGER.info(f'Image sizes {imgsz} train, {imgsz} val\n'
            f'Using {train_loader.num_workers} dataloader workers\n'
            f"Logging results to {colorstr('bold', save_dir)}\n"
            f'Starting training for {epochs} epochs...')


# print("GPU Usage after emptying the cache")
# torch.cuda.empty_cache()
# gpu_usage()

times = defaultdict(float)
epoch_times = {epoch: defaultdict(float) for epoch in range(start_epoch, epochs)}
batch_times = {i: defaultdict(float) for i in range(1 + np.max(dataset.batch))}
epoch_batch_times = {epoch: {i: defaultdict(float) for i in batch_times} for epoch in epoch_times}
for epoch in range(start_epoch, epochs):  # epoch ------------------------------------------------------------------
#     print(f"*** GPU Usage before epoch {epoch} in {{{start_epoch}, ..., {epochs-1}}}")
#     gpu_usage()
    LBs = []
    model.train()
    # if epoch == start_epoch:
    #     pred_bcc = pred0_bcc

    # Update image weights (optional, single-GPU only)
    if opt.image_weights:
        cw = model.class_weights.cpu().numpy() * (1 - maps) ** 2 / nc  # class weights
        iw = labels_to_image_weights(dataset.labels, nc=nc, class_weights=cw)  # image weights
        dataset.indices = random.choices(range(dataset.n), weights=iw, k=dataset.n)  # rand weighted idx

    # Update mosaic border (optional)
    # b = int(random.uniform(0.25 * imgsz, 0.75 * imgsz + gs) // gs * gs)
    # dataset.mosaic_border = [b - imgsz, -b]  # height, width borders

    # GOVIND0: Find mean losses by comparing pred with `qtargets`, not `targets`. Look for pointer GOVIND1
    mloss = torch.zeros(3, device=device)  # mean losses
    if RANK != -1:
        train_loader.sampler.set_epoch(epoch)
    pbar = enumerate(train_loader)
    LOGGER.info(('\n' + '%10s' * 7) % ('Epoch', 'gpu_mem', 'box', 'obj', 'cls', 'labels', 'img_size'))
    if RANK in [-1, 0]:
        pbar = tqdm(pbar, total=nb)  # progress bar
    optimizer.zero_grad()

    # YOLOBCC
    # qtargets, pcm['variational'], lb = VBi_yolo(cstargets_bcc, pred_bcc, pcm['variational'], pcm['prior'])
    for i, (imgs, targets, paths, _) in pbar:  # batch -------------------------------------------------------------
        batch_cstargets_bcc = (cstargets_bcc[dataset.batch == i]).to(device)
        # YOLOBCC
        # batch_qtargets = qtargets
        ni = i + nb * epoch  # number integrated batches (since train start)
        imgs = imgs.to(device, non_blocking=True).float() / 255.0  # uint8 to float32, 0-255 to 0.0-1.0

        # Warmup
        if ni <= nw:
            xi = [0, nw]  # x interp
            # compute_loss.gr = np.interp(ni, xi, [0.0, 1.0])  # iou loss ratio (obj_loss = 1.0 or iou)
            accumulate = max(1, np.interp(ni, xi, [1, nbs / batch_size]).round())
            for j, x in enumerate(optimizer.param_groups):
                # bias lr falls from 0.1 to lr0, all other lrs rise from 0.0 to lr0
                x['lr'] = np.interp(ni, xi, [hyp['warmup_bias_lr'] if j == 2 else 0.0, x['initial_lr'] * lf(epoch)])
                if 'momentum' in x:
                    x['momentum'] = np.interp(ni, xi, [hyp['warmup_momentum'], hyp['momentum']])

        # Multi-scale
        if opt.multi_scale:
            sz = random.randrange(imgsz * 0.5, imgsz * 1.5 + gs) // gs * gs  # size
            sf = sz / max(imgs.shape[2:])  # scale factor
            if sf != 1:
                ns = [math.ceil(x * sf / gs) * gs for x in imgs.shape[2:]]  # new shape (stretched to gs-multiple)
                imgs = nn.functional.interpolate(imgs, size=ns, mode='bilinear', align_corners=False)

        # Forward
        with amp.autocast(enabled=cuda):
            model.eval()

            t1 = timer()
            batch_pred_yolo = nn_predict(model, imgs, imgsz, transform_format_flag=False)
            t2 = timer()
            delta_t = round(t2 - t1, 2)
            epoch_batch_times[epoch][i]['nn_predict'] += delta_t
            epoch_times[epoch]['nn_predict'] += delta_t
            batch_times[i]['nn_predict'] += delta_t
            times['nn_predict'] += delta_t

            t1 = timer()
            batch_pred_bcc, batch_pred_yolo_wh = yolo2bcc_new(batch_pred_yolo, imgsz)
            t2 = timer()
            delta_t = round(t2 - t1, 2)
            epoch_batch_times[epoch][i]['yolo2bcc_new'] += delta_t
            epoch_times[epoch]['yolo2bcc_new'] += delta_t
            batch_times[i]['yolo2bcc_new'] += delta_t
            times['yolo2bcc_new'] += delta_t

            t1 = timer()
            batch_qtargets, batch_pcm['variational'], batch_lb = VBi_yolo(batch_cstargets_bcc, batch_pred_bcc, batch_pcm['variational'], batch_pcm['prior'], torchMode = torchMode, device=device)
            t2 = timer()
            delta_t = round(t2 - t1, 2)
            epoch_batch_times[epoch][i]['VBi_yolo'] += delta_t
            epoch_times[epoch]['VBi_yolo'] += delta_t
            batch_times[i]['VBi_yolo'] += delta_t
            times['VBi_yolo'] += delta_t

            LBs.append(batch_lb)
            
            break
            
            t1 = timer()
            batch_qtargets_yolo = qt2yolo(batch_qtargets, grid_ratios, n_anchor_choices, batch_pred_yolo_wh, torchMode = torchMode, device=device).half().float()
            t2 = timer()
            delta_t = round(t2 - t1, 2)
            epoch_batch_times[epoch][i]['qt2yolo'] += delta_t
            epoch_times[epoch]['qt2yolo'] += delta_t
            batch_times[i]['qt2yolo'] += delta_t
            times['qt2yolo'] += delta_t
            # pred_bcc = convert_yolo2bcc(pred_yolo.cpu().detach().numpy(), n_anchor_choices, nc, grid_ratios, intermediate_yolo_mode=True)

            model.train()

            t1 = timer()
            pred = model(imgs)  # forward
            t2 = timer()
            delta_t = round(t2 - t1, 2)
            epoch_batch_times[epoch][i]['pred'] += delta_t
            epoch_times[epoch]['pred'] += delta_t
            batch_times[i]['pred'] += delta_t
            times['pred'] += delta_t

            # GOVIND1: Don't compare w.r.t. targets, but w.r.t. qtargets (generated by BCC)
            # YOLOBCC
            # loss, loss_items = compute_loss(pred, targets)  # loss scaled by batch_size
            t1 = timer()
            loss, loss_items = compute_loss(pred, batch_qtargets_yolo)
            t2 = timer()
            delta_t = round(t2 - t1, 2)
            epoch_batch_times[epoch][i]['compute_loss'] += delta_t
            epoch_times[epoch]['compute_loss'] += delta_t
            batch_times[i]['compute_loss'] += delta_t
            times['compute_loss'] += delta_t

            if RANK != -1:
                loss *= WORLD_SIZE  # gradient averaged between devices in DDP mode
            if opt.quad:
                loss *= 4.
    break
        

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, mosaic=1.0, mixup=0.0, copy_paste=0.0
TensorBoard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
wandb: Currently logged in as: g-sharma (use `wandb login --relogin` to force relogin)


Overriding model.yaml nc=80 with nc=2

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Focus                     [3, 32, 3]                    
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  3    156928  models.common.C3                        [128, 128, 3]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]             

Plotting labels... 


Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/train/exp
Starting training for 300 epochs...

     Epoch   gpu_mem       box       obj       cls    labels  img_size
  0%|          | 0/1 [00:00<?, ?it/s]


autoanchor: Analyzing anchors... anchors/target = 5.25, Best Possible Recall (BPR) = 1.0000


  0%|          | 0/1 [00:00<?, ?it/s]


In [5]:
batch_qtargets.shape, grid_ratios, n_anchor_choices, batch_pred_yolo_wh.shape, torchMode, device

(torch.Size([2, 25200, 2]),
 array([     0.0125,       0.025,        0.05], dtype=float32),
 3,
 torch.Size([2, 25200, 2]),
 True,
 device(type='cpu'))

In [18]:
from label_converter import qt2yolo_optimized
qt_yolo1 = qt2yolo(batch_qtargets, grid_ratios, n_anchor_choices, batch_pred_yolo_wh, torchMode = torchMode, device=device).half().float()
with torch.no_grad():
    qt_yolo2 = qt2yolo_optimized(batch_qtargets, grid_ratios, n_anchor_choices, batch_pred_yolo_wh, torchMode = torchMode).half().float()

In [17]:
qt_yolo2

tensor([[0.00000, 0.00000, 0.00625, 0.00625, 0.02138, 0.00521],
        [0.00000, 0.00000, 0.01875, 0.00625, 0.02017, 0.01564],
        [0.00000, 0.00000, 0.03125, 0.00625, 0.01668, 0.01203],
        ...,
        [1.00000, 1.00000, 0.87500, 0.97510, 0.58447, 0.58301],
        [1.00000, 1.00000, 0.92480, 0.97510, 0.53223, 0.53613],
        [1.00000, 1.00000, 0.97510, 0.97510, 0.62109, 0.55029]], grad_fn=<CopyBackwards>)

In [19]:
torch.equal(qt_yolo1, qt_yolo2)

True

In [31]:
batch_qtargets.shape, batch_qtargets

(torch.Size([2, 25200, 2]),
 tensor([[[0.94075, 0.05925],
          [0.92852, 0.07148],
          [0.91696, 0.08304],
          ...,
          [0.42612, 0.57388],
          [0.43887, 0.56113],
          [0.44963, 0.55037]],
 
         [[0.93847, 0.06153],
          [0.92252, 0.07748],
          [0.91223, 0.08777],
          ...,
          [0.42407, 0.57593],
          [0.44052, 0.55948],
          [0.44128, 0.55872]]]))

In [150]:
entropy_thres = 0.75
p = batch_qtargets.reshape(qt_yolo.shape[0], 2)
(-p*p.log2()).sum(axis=1) <= entropy_thres

tensor([ True,  True,  True,  ..., False, False, False])

In [143]:
from matplotlib import pyplot as plt
%matplotlib inline

# plt.hist((-p*p.log2()).sum(axis=1).cpu().detach().numpy())

In [141]:
p = np.array([[x/10, 1-x/10] for x in range(10)])
p, (-p*np.log2(p)).sum(axis=1)

<ipython-input-141-d49e884fc16f>:2: RuntimeWarning: divide by zero encountered in log2
  p, (-p*np.log2(p)).sum(axis=1)
<ipython-input-141-d49e884fc16f>:2: RuntimeWarning: invalid value encountered in multiply
  p, (-p*np.log2(p)).sum(axis=1)


(array([[          0,           1],
        [        0.1,         0.9],
        [        0.2,         0.8],
        [        0.3,         0.7],
        [        0.4,         0.6],
        [        0.5,         0.5],
        [        0.6,         0.4],
        [        0.7,         0.3],
        [        0.8,         0.2],
        [        0.9,         0.1]]),
 array([        nan,       0.469,     0.72193,     0.88129,     0.97095,           1,     0.97095,     0.88129,     0.72193,       0.469]))

In [162]:
qt.shape, qt, conf.shape

(torch.Size([2, 25200, 2]),
 tensor([[[0.94075, 0.05925],
          [0.92852, 0.07148],
          [0.91696, 0.08304],
          ...,
          [0.42612, 0.57388],
          [0.43887, 0.56113],
          [0.44963, 0.55037]],
 
         [[0.93847, 0.06153],
          [0.92252, 0.07748],
          [0.91223, 0.08777],
          ...,
          [0.42407, 0.57593],
          [0.44052, 0.55948],
          [0.44128, 0.55872]]]),
 torch.Size([2, 25200]))

In [160]:
qt_yolo.shape, qt_yolo

(torch.Size([50400, 6]),
 tensor([[0.00000, 0.00000, 0.00625, 0.00625, 0.02138, 0.00521],
         [0.00000, 0.00000, 0.01875, 0.00625, 0.02017, 0.01564],
         [0.00000, 0.00000, 0.03125, 0.00625, 0.01668, 0.01203],
         ...,
         [1.00000, 1.00000, 0.87500, 0.97510, 0.58447, 0.58301],
         [1.00000, 1.00000, 0.92480, 0.97510, 0.53223, 0.53613],
         [1.00000, 1.00000, 0.97510, 0.97510, 0.62109, 0.55029]], grad_fn=<CopyBackwards>))

In [105]:
# high-conf-entropy
conf_thres = 0.01
qt_yolo[(conf >= conf_thres).reshape(qt_yolo.shape[0])]

tensor([[0.00000, 0.00000, 0.05624, 0.06873, 0.00578, 0.05725],
        [0.00000, 0.00000, 0.06873, 0.06873, 0.00478, 0.05463],
        [0.00000, 0.00000, 0.08124, 0.06873, 0.00408, 0.04559],
        ...,
        [1.00000, 1.00000, 0.87500, 0.97510, 0.58447, 0.58301],
        [1.00000, 1.00000, 0.92480, 0.97510, 0.53223, 0.53613],
        [1.00000, 1.00000, 0.97510, 0.97510, 0.62109, 0.55029]], grad_fn=<IndexBackward>)

In [92]:
count_thres = 10
# top_indices = conf.reshape(qt_yolo.shape[0]).sort(descending=True).indices[:count_thres]
# qt_yolo[top_indices, :]
n_images = conf.shape[0]
n_boxes = conf.shape[1]
top_indices = conf.sort(descending=True).indices[:, :count_thres]
indices = top_indices + torch.tensor([[i*n_boxes for i in range(n_images)]]).transpose(1, 0)
indices = indices.reshape(indices.shape[0]*indices.shape[1])
return qt_yolo[indices, :]

tensor([[0.00000, 1.00000, 0.47510, 0.12500, 0.46851, 0.19043],
        [0.00000, 1.00000, 0.37500, 0.17505, 0.50000, 0.16528],
        [0.00000, 1.00000, 0.32495, 0.17505, 0.60400, 0.22375],
        [0.00000, 0.00000, 0.42505, 0.12500, 0.31421, 0.17004],
        [0.00000, 0.00000, 0.42505, 0.17505, 0.26562, 0.14038],
        [0.00000, 1.00000, 0.87500, 0.82520, 0.18457, 0.21130],
        [0.00000, 1.00000, 0.47510, 0.17505, 0.33813, 0.19556],
        [0.00000, 0.00000, 0.37500, 0.12500, 0.50928, 0.20435],
        [0.00000, 1.00000, 0.42505, 0.32495, 0.51318, 0.10480],
        [0.00000, 1.00000, 0.92480, 0.82520, 0.16931, 0.17725],
        [1.00000, 1.00000, 0.87500, 0.77490, 0.71338, 0.39648],
        [1.00000, 1.00000, 0.17505, 0.82520, 0.60938, 0.45947],
        [1.00000, 1.00000, 0.22498, 0.82520, 0.78906, 0.44482],
        [1.00000, 1.00000, 0.07501, 0.82520, 0.69336, 0.29785],
        [1.00000, 1.00000, 0.32495, 0.82520, 0.87354, 0.32422],
        [1.00000, 1.00000, 0.02499, 0.82

In [81]:
n_images = conf.shape[0]
n_boxes = conf.shape[1]
[i*n_boxes for i in range(n_images)]

[0, 25200]

In [56]:
qt_yolo = batch_qtargets_yolo
conf = batch_pred_yolo[:, :, 4]

conf_thres = 0.01
n_images = conf.shape[0]
# for i in range(n_images):
i = 0
from collections import Counter
qt_yolo.shape, qt_yolo[(conf > conf_thres).reshape(qt_yolo.shape[0])].shape

(torch.Size([50400, 6]), torch.Size([2932, 6]))

In [29]:
batch_qtargets_yolo = qt2yolo_optimized(batch_qtargets, grid_ratios, n_anchor_choices, batch_pred_yolo_wh, torchMode = torchMode).half().float()

In [16]:
MAX_GRID_CELLS = 99999
BACKGROUND_CLASS_ID = -1

def find_grid_center(grid_x_frac, grid_y_frac, gc, mode='row-major'):
    if grid_x_frac == 0 or grid_y_frac == 0:
        raise Exception(
            "Cannot divide into infinite grids. Ensure grid fractions are > 0.")
    n_x_cells, n_y_cells = np.ceil(1 / grid_x_frac).astype(int), np.ceil(1 / grid_y_frac).astype(int)
    if n_x_cells*n_y_cells > MAX_GRID_CELLS:
        raise Exception("Grid cells maxed out.")
    if mode.startswith('row'):
        x_cell_id, y_cell_id = gc % n_x_cells, np.floor(gc / n_x_cells).astype(int)
    elif mode.startswith('col'):
        x_cell_id, y_cell_id = np.floor(gc / n_y_cells).astype(int), gc % n_y_cells
    else:
        raise Exception("Incorrect mode for finding grid center.")
    x = x_cell_id*grid_x_frac + grid_x_frac/2
    y = y_cell_id*grid_y_frac + grid_y_frac/2
    return x, y

In [21]:
xy.device

device(type='cpu')

In [20]:
qt = batch_qtargets
G = grid_ratios
Na = n_anchor_choices
wh_yolo = batch_pred_yolo_wh

Ng = G.shape[0]
num_images = qt.shape[0]
y_bcc = []
# for i in range(num_images):
i = 0
cs = torch.argmax(qt[i, :], 1)
st = 0
# for g in range(Ng):
g = 0
g_frac = 0.25#G[g]
S_g = np.ceil(1/g_frac).astype(int)
n_cells = S_g*S_g
# for a in range(Na):
a = 0
z = torch.linspace(g_frac/2, 1-g_frac/2, S_g).repeat(S_g, 1).unsqueeze(-1)
xy = torch.cat((z.permute(1, 0, 2), z), 2).permute(1, 0, 2).reshape(n_cells, 2)
wh = wh_yolo[i][st:st+n_cells]
c = cs[st:st+n_cells]
icxywh = torch.cat((i*torch.ones(n_cells, 1), c.unsqueeze(-1), xy, wh), 1)
y_bcc.append(icxywh)
st += n_cells
# torch.cat(y_bcc).shape
icxywh

tensor([[0.00000, 0.00000, 0.12500, 0.12500, 0.02138, 0.00521],
        [0.00000, 0.00000, 0.37500, 0.12500, 0.02017, 0.01564],
        [0.00000, 0.00000, 0.62500, 0.12500, 0.01668, 0.01203],
        [0.00000, 0.00000, 0.87500, 0.12500, 0.01415, 0.01787],
        [0.00000, 0.00000, 0.12500, 0.37500, 0.01383, 0.02351],
        [0.00000, 0.00000, 0.37500, 0.37500, 0.01384, 0.02454],
        [0.00000, 0.00000, 0.62500, 0.37500, 0.01687, 0.02681],
        [0.00000, 0.00000, 0.87500, 0.37500, 0.01815, 0.02719],
        [0.00000, 0.00000, 0.12500, 0.62500, 0.01826, 0.02552],
        [0.00000, 0.00000, 0.37500, 0.62500, 0.01929, 0.02550],
        [0.00000, 0.00000, 0.62500, 0.62500, 0.01783, 0.02484],
        [0.00000, 0.00000, 0.87500, 0.62500, 0.01832, 0.02437],
        [0.00000, 0.00000, 0.12500, 0.87500, 0.01727, 0.02172],
        [0.00000, 0.00000, 0.37500, 0.87500, 0.01810, 0.02072],
        [0.00000, 0.00000, 0.62500, 0.87500, 0.01606, 0.01922],
        [0.00000, 0.00000, 0.87500, 0.87

tensor([[0.00000, 0.00000, 0.00625, 0.00625, 0.02138, 0.00521],
        [0.00000, 0.00000, 0.00625, 0.01875, 0.02017, 0.01564],
        [0.00000, 0.00000, 0.00625, 0.03125, 0.01668, 0.01203],
        ...,
        [0.00000, 0.00000, 0.99375, 0.96875, 0.02559, 0.00252],
        [0.00000, 0.00000, 0.99375, 0.98125, 0.02765, 0.00269],
        [0.00000, 0.00000, 0.99375, 0.99375, 0.01773, 0.00155]], grad_fn=<CatBackward>)

In [29]:
Counter(torch.argmax(qt, 2)[0].cpu().detach().numpy()), Counter(torch.argmax(qt, 2)[1].cpu().detach().numpy())

(Counter({0: 20853, 1: 4347}), Counter({0: 21489, 1: 3711}))

In [27]:
from collections import Counter
Counter(torch.argmax(qt[0, :], 1).cpu().detach().numpy()), Counter(torch.argmax(qt[1, :], 1).cpu().detach().numpy())

(Counter({0: 20853, 1: 4347}), Counter({0: 21489, 1: 3711}))

In [136]:
qt = batch_qtargets
G = grid_ratios
Na = n_anchor_choices
wh_yolo = batch_pred_yolo_wh

Ng = G.shape[0]
y_bcc = []
num_images = qt.shape[0]
for i in range(num_images):
    effective_id = 0
    cs = torch.argmax(qt[i, :], 1) if torchMode else np.argmax(qt[i, :], 1)
    for g in range(Ng):
        g_frac = G[g]
        S_g = np.ceil(1/g_frac).astype(int)
        for a in range(Na):
            for gc in range(S_g*S_g):
                x, y = find_grid_center(g_frac, g_frac, gc)
                w, h = wh_yolo[i][effective_id]
                c = cs[effective_id]
                y_bcc.append([i, c, x, y, w, h])
                effective_id += 1
torch.tensor(y_bcc).to(device)

tensor([[0.00000, 0.00000, 0.00625, 0.00625, 0.02138, 0.00521],
        [0.00000, 0.00000, 0.01875, 0.00625, 0.02017, 0.01564],
        [0.00000, 0.00000, 0.03125, 0.00625, 0.01668, 0.01203],
        ...,
        [1.00000, 1.00000, 0.87500, 0.97500, 0.58438, 0.58290],
        [1.00000, 1.00000, 0.92500, 0.97500, 0.53225, 0.53605],
        [1.00000, 1.00000, 0.97500, 0.97500, 0.62085, 0.55034]], dtype=torch.float64)

In [137]:
torch.tensor(y_bcc), torch.cat(y_bcc).shape

(torch.Size([50400, 6]),
 tensor([[0.00000, 0.00000, 0.00625, 0.00625, 0.02138, 0.00521],
         [0.00000, 0.00000, 0.01875, 0.00625, 0.02017, 0.01564],
         [0.00000, 0.00000, 0.03125, 0.00625, 0.01668, 0.01203],
         ...,
         [1.00000, 1.00000, 0.87500, 0.97500, 0.58438, 0.58290],
         [1.00000, 1.00000, 0.92500, 0.97500, 0.53225, 0.53605],
         [1.00000, 1.00000, 0.97500, 0.97500, 0.62085, 0.55034]], dtype=torch.float64))

In [ ]:
qtargets, pcm_var, lb = VBi_yolo(cstargets_bcc, pred_bcc, pcm['variational'], pcm['prior'])
qtargets, pcm_var, lb

In [274]:

# part of computing loss
def logB_from_Dirichlet_parameters(alpha):
    torchMode = torch.is_tensor(alpha)
    if torchMode:
        base_lib = torch
        gammaln_fn = torch.special.gammaln
        simple_transpose = lambda x: torch.transpose(x, 0, 1)
    else:
        base_lib = np
        gammaln_fn = ss.gammaln
        simple_transpose = np.transpose
    logB = base_lib.sum(gammaln_fn(alpha)) - gammaln_fn(base_lib.sum(alpha))

    return logB


def expected_log_Dirichlet_parameters(param):
    torchMode = torch.is_tensor(param)
    if torchMode:
        base_lib = torch
        digamma_fn = torch.digamma
        simple_transpose = lambda x: torch.transpose(x, 0, 1)
    else:
        base_lib = np
        digamma_fn = ss.psi
        simple_transpose = np.transpose
        
    size = param.shape
    result = base_lib.zeros_like(param)

    if len(size) == 1:
        result = digamma_fn(param) - digamma_fn(base_lib.sum(param))
    elif len(size) == 2:  # when we take A_0 for everyone
        result = digamma_fn(param) - simple_transpose(base_lib.tile(digamma_fn(base_lib.sum(param, 1)), (size[1], 1)))
    elif len(size) == 3:  # most of time for posterior cm
        for i in range(size[2]):
            result[:, :, i] = digamma_fn(param[:, :, i]) - \
                              simple_transpose(base_lib.tile(digamma_fn(base_lib.sum(param[:, :, i], 1)), (size[1], 1)))
    else:
        raise Exception('param can have no more than 3 dimensions')
    return result


def expected_true_labels(X, nn_output, ElogPi_volunteer):
    def max_fun(t1, t2):
        if not torch.is_tensor(t1):
            t1 = torch.tensor(t1)
        if not torch.is_tensor(t2):
            t2 = torch.tensor(t2)
        return torch.max(t1, t2)
    torchMode = all([torch.is_tensor(x) for x in [X, nn_output, ElogPi_volunteer]])
    if torchMode:
        torch_invert_dims = lambda x: torch.permute(x, tuple(range(x.ndim)[::-1]))
        base_lib = torch
        digamma_fn = torch.digamma
        copy_fn = lambda x: x.clone().detach()
        simple_transpose = torch_invert_dims
        maxwithdim_fn = lambda x, d: torch.max(x, d).values
        maximum_fn = lambda t1, t2: max_fun(t1, t2)
    else:
        base_lib = np
        digamma_fn = ss.psi
        copy_fn = np.copy
        simple_transpose = np.transpose
        maxwithdim_fn = np.max
        maximum_fn = np.maximum

    I, U, K = X.shape  # I = no. of image, U = no. of anchor boxes in total, K = no. of volunteers
    M = ElogPi_volunteer.shape[0]  # M = Number of classes
    N = ElogPi_volunteer.shape[1]  # N = Number of classes used by volunteers

    rho = copy_fn(nn_output)  # I x U x M logits
    # eq. 12:
    for k in range(K):
        inds = base_lib.where(X[:, :, k] > -1)  # rule out missing values
        rho[inds[0], inds[1], :] += simple_transpose(
            ElogPi_volunteer[:, base_lib.squeeze(X[inds[0], inds[1], k]), k])

    # normalisation: (minus the max of each anchor)
    rho -= simple_transpose(base_lib.tile(simple_transpose(maxwithdim_fn(rho, 2)), (M, 1, 1)))

    # # eq. 11:
    q_t = base_lib.exp(rho) / maximum_fn(1e-60, simple_transpose(base_lib.tile(simple_transpose(base_lib.sum(base_lib.exp(rho), 2)), (M, 1, 1))))
    q_t = maximum_fn(1e-60, q_t)

    # partial of eq. 8: (right side 2nd term)
    f_iu = base_lib.zeros((M, N, K), dtype=base_lib.float64)
    for k in range(K):
        for n in range(N):
            ids0 = base_lib.where(X[:, :, k] == n)[0]
            ids1 = base_lib.where(X[:, :, k] == n)[1]
            f_iu[:, n, k] = base_lib.sum(q_t[ids0, ids1, :], 0)
    rho.shape, rho
    return q_t, f_iu, rho
# dim: (I x U x M), (M x N x K), (I x U x M)


# eq. 8:
def update_alpha_volunteers(alpha0_volunteers, f_iu):
    torchMode = all([torch.is_tensor(x) for x in [alpha0_volunteers, f_iu]])
    if torchMode:
        base_lib = torch
    else:
        base_lib = np
    K = alpha0_volunteers.shape[2]
    alpha_volunteers = base_lib.zeros_like(alpha0_volunteers)

    for k in range(K):
        alpha_volunteers[:, :, k] = alpha0_volunteers[:, :, k] + f_iu[:, :, k]

    return alpha_volunteers


def compute_lower_bound_likelihood(alpha0_volunteers, alpha_volunteers, q_t, rho, nn_output):
    torchMode = all([torch.is_tensor(x) for x in [alpha0_volunteers, alpha_volunteers, q_t, rho, nn_output]])
    if torchMode:
        base_lib = torch
    else:
        base_lib = np
    
    W = alpha0_volunteers.shape[2]

    ll_pi_worker = 0
    for w in range(W):
        ll_pi_worker -= base_lib.sum(logB_from_Dirichlet_parameters(alpha0_volunteers[:, :, w]) -
                                             logB_from_Dirichlet_parameters(alpha_volunteers[:, :, w]))

    ll_t = -base_lib.sum(q_t * rho) + base_lib.sum(base_lib.log(base_lib.sum(base_lib.exp(rho), axis=1)), axis=0)

    ll_nn = base_lib.sum(q_t * nn_output) - base_lib.sum(base_lib.log(base_lib.sum(base_lib.exp(nn_output), axis=1)), axis=0)

    ll = ll_pi_worker + ll_t + ll_nn  # VB lower bound

    return base_lib.sum(ll)

In [275]:
def np2pt(np, device):
    return torch.tensor(np).to(device)

def pt2np(pt):
    return pt.cpu().detach().numpy()

## NumPy

In [276]:
import scipy.special as ss
import numpy as np
# import pdb

X, nn_output, alpha_volunteers, alpha0_volunteers = cstargets_bcc, pred_bcc.cpu().detach().numpy(), \
    pcm['variational'], pcm['prior']

# def VB_iteration(X, nn_output, alpha_volunteers, alpha0_volunteers):
# pdb.set_trace()
ElogPi_volunteer = expected_log_Dirichlet_parameters(alpha_volunteers)

# q_t
q_t, Njl, rho = expected_true_labels(X, nn_output, ElogPi_volunteer)

# q_pi_workers
# alpha_volunteers = update_alpha_volunteers(alpha0_volunteers, Njl)

# # Low bound
# lower_bound_likelihood = compute_lower_bound_likelihood(alpha0_volunteers, alpha_volunteers, \
#                                                         q_t, rho, nn_output)

# qtargets, pcm_var, lb = q_t, alpha_volunteers, lower_bound_likelihood

q_t, Njl, rho

(array([[[    0.93847,    0.061533],
         [    0.92252,    0.077477],
         [    0.91223,    0.087769],
         ...,
         [    0.42407,     0.57593],
         [    0.44052,     0.55948],
         [    0.44128,     0.55872]],
 
        [[    0.94075,    0.059253],
         [    0.92852,     0.07148],
         [    0.91696,    0.083039],
         ...,
         [    0.42612,     0.57388],
         [    0.43887,     0.56113],
         [    0.44963,     0.55037]]], dtype=float32),
 array([[[     29.085,       11.94,      35.662,      30.173],
         [          0,      14.694,           0,           0]],
 
        [[     15.915,      6.0599,      18.338,      14.827],
         [          0,      12.306,           0,           0]]]),
 array([[[          0,     -2.7247],
         [          0,     -2.4771],
         [          0,     -2.3412],
         ...,
         [   -0.30607,           0],
         [   -0.23904,           0],
         [   -0.23595,           0]],
 
        [[

## PyTorch

In [277]:
X, nn_output, alpha_volunteers, alpha0_volunteers = torch.tensor(cstargets_bcc), pred_bcc, \
    torch.tensor(pcm['variational']), torch.tensor(pcm['prior'])

# def VB_iteration_pytorch(X, nn_output, alpha_volunteers, alpha0_volunteers):

# pdb.set_trace()
ElogPi_volunteer = expected_log_Dirichlet_parameters(alpha_volunteers)

# q_t
q_t, Njl, rho = expected_true_labels(X, nn_output, ElogPi_volunteer)

# q_pi_workers
# alpha_volunteers = update_alpha_volunteers(alpha0_volunteers, Njl)

# # Low bound
# lower_bound_likelihood = compute_lower_bound_likelihood(alpha0_volunteers, alpha_volunteers, \
#                                                         q_t, rho, nn_output)

# qtargets, pcm_var, lb = q_t, alpha_volunteers, lower_bound_likelihood

q_t, Njl, rho

(tensor([[[0.93847, 0.06153],
          [0.92252, 0.07748],
          [0.91223, 0.08777],
          ...,
          [0.42407, 0.57593],
          [0.44052, 0.55948],
          [0.44128, 0.55872]],
 
         [[0.94075, 0.05925],
          [0.92852, 0.07148],
          [0.91696, 0.08304],
          ...,
          [0.42612, 0.57388],
          [0.43887, 0.56113],
          [0.44963, 0.55037]]]),
 tensor([[[29.08494, 11.94005, 35.66164, 30.17300],
          [ 0.00000, 14.69351,  0.00000,  0.00000]],
 
         [[15.91506,  6.05995, 18.33836, 14.82700],
          [ 0.00000, 12.30649,  0.00000,  0.00000]]], dtype=torch.float64),
 tensor([[[ 0.00000, -2.72467],
          [ 0.00000, -2.47713],
          [ 0.00000, -2.34119],
          ...,
          [-0.30607,  0.00000],
          [-0.23904,  0.00000],
          [-0.23595,  0.00000]],
 
         [[ 0.00000, -2.76486],
          [ 0.00000, -2.56418],
          [ 0.00000, -2.40175],
          ...,
          [-0.29771,  0.00000],
          [-0.24

In [5]:
model.eval()
pred_yolo = nn_predict(model, imgs, imgsz, transform_format_flag=False)
pred_yolo.shape, pred_yolo

(torch.Size([2, 25200, 7]),
 tensor([[[3.62112e-03, 9.27925e-03, 1.36846e+01,  ..., 1.49550e-03, 8.22292e-01, 5.17919e-02],
          [1.59202e-02, 7.25159e-03, 1.29100e+01,  ..., 1.89948e-03, 8.26544e-01, 6.36292e-02],
          [2.91756e-02, 7.79393e-03, 1.06776e+01,  ..., 1.34058e-03, 7.16163e-01, 6.48550e-02],
          ...,
          [8.87906e-01, 9.82400e-01, 3.69486e+02,  ..., 1.89169e-02, 3.72366e-01, 5.01492e-01],
          [9.36504e-01, 9.79430e-01, 3.28244e+02,  ..., 1.73013e-02, 3.57602e-01, 4.57227e-01],
          [9.83267e-01, 9.79760e-01, 3.76296e+02,  ..., 1.72529e-02, 3.72640e-01, 4.56131e-01]],
 
         [[3.80318e-03, 8.85521e-03, 1.23594e+01,  ..., 1.40285e-03, 8.24831e-01, 5.40825e-02],
          [1.61285e-02, 6.41171e-03, 1.08387e+01,  ..., 1.87619e-03, 8.26284e-01, 6.93943e-02],
          [2.98106e-02, 6.88588e-03, 8.18122e+00,  ..., 1.46679e-03, 7.17260e-01, 6.90101e-02],
          ...,
          [8.84591e-01, 9.75536e-01, 3.74005e+02,  ..., 1.49167e-02, 4.0546

In [8]:
from label_converter import yolo2bcc_new
pred_bcc, pred_yolo_wh = yolo2bcc_new(pred_yolo)
model.train()
pred_yolo_wh = pred_yolo_wh.cpu().detach().numpy()
pred = model(imgs)  # forward

In [9]:
qtargets, pcm_v, lb = VBi_yolo(cstargets_bcc, pred_bcc.detach().numpy(), pcm['variational'], pcm['prior'])

In [10]:
qtargets.shape, pred_yolo_wh.shape

((2, 25200, 2), (2, 25200, 2))

In [11]:
!cat ../../datasets/toy/labels/train/train*.txt

1 0.7167199148029819 0.2063380281690141 0.10862619808306709 0.18450704225352113
1 0.6629392971246006 0.6330985915492958 0.10543130990415335 0.1732394366197183
0 0.1847710330138445 0.6098591549295774 0.13099041533546327 0.15774647887323945
1 0.5213219616204691 0.30277777777777776 0.31544562899786777 0.41095555555555563


In [12]:
targets.shape, targets

(torch.Size([4, 6]),
 tensor([[0.00000, 1.00000, 0.52132, 0.34869, 0.31545, 0.31528],
         [1.00000, 1.00000, 0.71672, 0.27838, 0.10863, 0.13925],
         [1.00000, 1.00000, 0.66294, 0.60045, 0.10543, 0.13074],
         [1.00000, 0.00000, 0.18477, 0.58291, 0.13099, 0.11905]]))

In [20]:
from label_converter import find_grid_center
def qt2yolo(qt):
    G = grid_ratios
    Ng = G.shape[0]
    Na = 3
    y_bcc = []
    num_images, num_grid_cells, _ = qt.shape
    for i in range(num_images):
        effective_id = 0
        cs = np.argmax(qt[i, :], 1)            
        for g in range(Ng):
            g_frac = G[g]
            S_g = np.ceil(1/g_frac).astype(int)
            for a in range(Na):
                for gc in range(S_g*S_g):
                    x, y = find_grid_center(g_frac, g_frac, gc)
                    w, h = pred_yolo_wh[i][effective_id]
                    c = cs[effective_id]
    #                 if (effective_id%100==0):
    #                     print('I =', i, ', g =', g, ', a =', a, ', gc =', gc, qt[i, effective_id, :], \
    #                           (round(x, 2), round(y, 2), round(w, 2), round(h, 2), c))
                    y_bcc.append([i, c, x, y, w, h])
                    effective_id += 1
    return np.array(y_bcc)

qt2yolo(qtargets)

array([[          0,           0,     0.00625,     0.00625,      13.685,      3.3365],
       [          0,           0,     0.01875,     0.00625,       12.91,      10.012],
       [          0,           0,     0.03125,     0.00625,      10.678,      7.7009],
       ...,
       [          1,           1,       0.875,       0.975,      374.01,      373.05],
       [          1,           1,       0.925,       0.975,      340.64,      343.07],
       [          1,           1,       0.975,       0.975,      397.34,      352.22]])

In [24]:
pred0_bcc

array([[[    0.97874,      2.2409],
        [     1.8676,    -0.97728],
        [    0.95009,    -0.15136],
        ...,
        [    0.79987,       1.579],
        [   -0.47848,     -1.3616],
        [    -1.1311,    -0.87371]],

       [[    0.45091,    -0.99932],
        [    -1.9487,     -1.0684],
        [   -0.68979,     0.19958],
        ...,
        [    0.14034,    0.040687],
        [    0.56578,     -2.6195],
        [   -0.79674,    -0.92146]]])

In [ ]:
from IPython.display import Image 
pil_img = Image(filename=os.path.join(data_dict['train'], 'train1.jpg'))
display(pil_img)

In [ ]:
targets

In [ ]:
z = targets[:,2:]
# z[:, :2] = z[:, :2] - z[:, 2:]/2
# z[:, 2:] = z[:, :2] + z[:, 2:]
z*x_size

In [ ]:
n_images, _, x_size, y_size = imgs.shape

In [ ]:
pred_bcc = torch.log(pred_yolo[:, ..., 5:]/pred_yolo[:,...,5:].sum(2).unsqueeze(-1))

In [ ]:
pred_bcc

In [ ]:
pred_yolo[:,...,5:].sum(2).unsqueeze(-1)

In [ ]:
batch_qtargets.shape, batch_qtarg

In [ ]:
convert_bcc2yolo():
bcc_labels = {'labels': batch_qtargets, 'Na': 3, 'Nc': 2, 'wh_map': }
G, Na, Nc, wh_map = [bcc_labels[x] for x in ['G', 'Na', 'Nc', 'wh_map']]
Ng = G.shape[0]
S = (1/G).astype(int)
C = list(range(Nc))
effective_id = 0
yolo_labels = []
for g in range(Ng):
    g_labels = []
    g_frac = G[g]
    cells_per_side = np.ceil(1/g_frac).astype(int)
    for a in range(Na):
        a_labels = []
        for gc in range(cells_per_side**2):  # grid-cells
            c = bcc_labels['labels'][effective_id]
            effective_id += 1
            if c == BACKGROUND_CLASS_ID:
                continue
            x, y = find_grid_center(g_frac, g_frac, gc)
            # print(effective_id, c)
            w, h = bcc_labels['wh_map'][(g, a, gc)]
            gc_label = [c, x, y, w, h]
            a_labels.append(gc_label)
        g_labels.append(a_labels)
    yolo_labels.append(g_labels)
if return_flattened:
    y = np.concatenate([np.concatenate(x) for x in yolo_labels])
else:
    y = np.array(yolo_labels)
return {'labels': y, 'G': G, 'Nc': Nc}

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib.patches as patches

fig, ax = plt.subplots()
imgplot = plt.imshow(np.transpose(imgs[1], (1, 2, 0)), cmap='gray')

for i in range(3, z.shape[0]):
    x, y, w, h = z[i]*x_size
    rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)

fig, ax = plt.subplots()
imgplot = plt.imshow(np.transpose(imgs[0], (1, 2, 0)), cmap='gray')

for i in range(3):
    x, y, w, h = z[i]*x_size
    rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)

In [ ]:
q_t, pcm_var, lb = VBi_yolo(cstargets_bcc, pred_bcc.cpu().detach().numpy(), pcm['variational'], pcm['prior'])

In [ ]:
lb

In [ ]:
z[:,...,5:]

In [ ]:
Counter(cstargets_bcc[1][:, 4])

In [ ]:
pred_bcc.shape

In [ ]:
pred_bcc = convert_yolo2bcc(pred_yolo.cpu().detach().numpy(), n_anchor_choices, nc, grid_ratios, intermediate_yolo_mode = True)
pred_bcc.shape, pred_bcc

In [ ]:
pred_yolo.shape, pred_bcc.shape

In [ ]:
from collections import Counter
Counter(pred_yolo[...,0].cpu().detach().numpy()[1]), Counter(pred_bcc[1])

In [ ]:
Counter(pred_yolo[0][:, 0].cpu().detach().numpy()), Counter(pred_bcc[0])

In [ ]:
pred_yolo[0, :4, :]

In [ ]:
y = pred_yolo.cpu().detach().numpy()
y.shape, y

In [ ]:
y.shape

In [ ]:
i = 0
effective_id = 0
points = {}
for g, gr in enumerate(grid_ratios):
    points[g] = {}
    cells_per_side = np.ceil(1/gr).astype(int)
    for a in range(3):
        points[g][a] = []
        for gc in range(cells_per_side**2):
            points[g][a].append(y[i, effective_id, [1,2]])
            effective_id += 1
        points[g][a] = np.array(points[g][a])
        print(f'(G{g}, A{a}): {points[g][a].shape[0]} points')

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
a = 0
for g in range(3):
    r = grid_ratios[g]
    # x1,  = grid_ratios[0], 0
    fig, ax = plt.subplots(figsize=(20, 20))

    ax.set_xticks(np.arange(0, 1+r, r))
    ax.set_yticks(np.arange(0, 1+r, r))
    # ax.set_xticklabels([0, 1])
    ax.set_yticklabels([round(1-j, 2) for j in ax.get_yticks()])
    ax.set_xlim([-3*r, 1+3*r])
    ax.set_ylim([-3*r, 1+3*r])
    # ax.set_xlim([-3*r, 0.1])
    # ax.set_ylim([1-(-3*r), 1-0.1])
    plt.plot(points[g][a][:,0], 1-points[g][a][:,1], linestyle=' ', marker="x", markersize=5)
    plt.grid(True)
    plt.show()
    fig.autofmt_xdate()